# Causes derrière la consommation de drogue 

Vous allez retouver sur ce notebook les différentes analyses effectuées dans l'optique de répondre à la problématique suivante :
- Peut-on prédire la consommation de drogue "dure" ou dangereuse selon la personnalité et l'environnement des jeunes (18-24) ?

URL to our dataset : https://archive.ics.uci.edu/ml/datasets/Drug+consumption+%28quantified%29

Le dataset que nous allons traité a été collecté en 2016. 
Le traitement de notre data passera par un travail de classification afin de déterminer si des profils ressortent dans notre dataset.

Poure ce faire, nous allons passer par les étapes suivantes :
- Description et exploration en surface de la data
- Data Cleaning 
- Data Wrangling (exploration plus approffondie)
- Visualisation de la data (maintenant qu'on a un jeu de données exploitables, on peut le visualiser)
- Action (ML models etc )

In [503]:
from mapping import *

---

## <p style = 'text-align: center;'>Chargement et Description des données</p> 

---

In [504]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

from IPython.display import display, HTML 

css = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(css))

In [505]:
# chargement des données
# noms des colonnes

data =  pd.read_csv('drug_consumption.csv', sep=",", header = None)
original_data = data.copy()


data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
1,2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,...,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
2,3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,...,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
3,4,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,...,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
4,5,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,...,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0


Nom de chaque colonne : 

In [506]:
data.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
           dtype='int64')

Nous allons renommer ces colonnes 

In [507]:
names_column = ['ID','age','gender','education','country','ethnicity','nscore','escore','oscore','ascore','csore','impulsive_bis11','sensation_ImpSS','alcohol','amphet','amyl','benzos','caff','cannabis','choc','coke','crack','ecstasy','heroin','ketamine','legalhigh','LSD','meth','mushrooms','nicotine','semer','VSabuse']

In [508]:
columns_dict = dict(zip(list(range(len(data.columns))),names_column))

data.rename(columns = columns_dict, inplace = True)

#data.rename( columns = names_column )

In [509]:
data.columns

Index(['ID', 'age', 'gender', 'education', 'country', 'ethnicity', 'nscore',
       'escore', 'oscore', 'ascore', 'csore', 'impulsive_bis11',
       'sensation_ImpSS', 'alcohol', 'amphet', 'amyl', 'benzos', 'caff',
       'cannabis', 'choc', 'coke', 'crack', 'ecstasy', 'heroin', 'ketamine',
       'legalhigh', 'LSD', 'meth', 'mushrooms', 'nicotine', 'semer',
       'VSabuse'],
      dtype='object')

In [510]:
print(data.describe)

<bound method NDFrame.describe of         ID      age   gender  education  country  ethnicity   nscore   escore  \
0        1  0.49788  0.48246   -0.05921  0.96082    0.12600  0.31287 -0.57545   
1        2 -0.07854 -0.48246    1.98437  0.96082   -0.31685 -0.67825  1.93886   
2        3  0.49788 -0.48246   -0.05921  0.96082   -0.31685 -0.46725  0.80523   
3        4 -0.95197  0.48246    1.16365  0.96082   -0.31685 -0.14882 -0.80615   
4        5  0.49788  0.48246    1.98437  0.96082   -0.31685  0.73545 -1.63340   
...    ...      ...      ...        ...      ...        ...      ...      ...   
1880  1884 -0.95197  0.48246   -0.61113 -0.57009   -0.31685 -1.19430  1.74091   
1881  1885 -0.95197 -0.48246   -0.61113 -0.57009   -0.31685 -0.24649  1.74091   
1882  1886 -0.07854  0.48246    0.45468 -0.57009   -0.31685  1.13281 -1.37639   
1883  1887 -0.95197  0.48246   -0.61113 -0.57009   -0.31685  0.91093 -1.92173   
1884  1888 -0.95197 -0.48246   -0.61113  0.21128   -0.31685 -0.46725  2.127

Nous allons travailler sur deux jeux de données : 
- data : jeu de données générales sur l'ensemble des ages 
- data_jeunes : jeu de données se focalisant sur les jeunes [18ans-25ans]

In [511]:
# description des données/ nombre d'invidus et nombre de caractéristiques
# 1er jeu de données
data.shape

(1885, 32)

- 1885 rows -> 1885 d'individus 
- 32 columns : 32 caractéristiques d'individus

In [512]:
# 2ème jeu de données
data_jeunes = data.loc[data['age'] == -0.95197]

In [513]:
data_jeunes.shape

(643, 32)

In [514]:
pourcentage_jeunes = (data_jeunes.shape[0]/data.shape[0])*100

In [515]:
print('il y a ',data_jeunes.shape[0],' jeunes sur 1885 individus soit : ',round(pourcentage_jeunes,2), '% de jeunes dans notre jeu de données')

il y a  643  jeunes sur 1885 individus soit :  34.11 % de jeunes dans notre jeu de données


In [516]:
# description des colonnes
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1885 entries, 0 to 1884
Data columns (total 32 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               1885 non-null   int64  
 1   age              1885 non-null   float64
 2   gender           1885 non-null   float64
 3   education        1885 non-null   float64
 4   country          1885 non-null   float64
 5   ethnicity        1885 non-null   float64
 6   nscore           1885 non-null   float64
 7   escore           1885 non-null   float64
 8   oscore           1885 non-null   float64
 9   ascore           1885 non-null   float64
 10  csore            1885 non-null   float64
 11  impulsive_bis11  1885 non-null   float64
 12  sensation_ImpSS  1885 non-null   float64
 13  alcohol          1885 non-null   object 
 14  amphet           1885 non-null   object 
 15  amyl             1885 non-null   object 
 16  benzos           1885 non-null   object 
 17  caff          

Incohérence au niveau des types compte tenu des variables avec lesquelles nous allons travailler.\
Exemple : Age devrait être un Integer pas un float et l'éthnicité qui est une variable à priori, nominale ne peut pas être Float non plus.

Nous allons dès lors essayer de mieux comprendre cers incohérences

Nous observons trois Grands types de colonnes : 
- Colonnes décrivant le profil de l'individu ( profil )<br />---> nous permettra d'en savoir plus sur son environnement de vie.

- Colonnes décrivant la personnalité de l'individu ( perso )<br />---> nous permettra d'en savoir plus sur ses traits de personnalité.

- Colonnes décrivant la fréquence de consommation de drogues ( drugs )<br />---> nous permettra de définir notre Target ( predicting the consommation is what we are searching for )


In [517]:
l_perso = list(range(6,13))
l_drugs = list(range(13,32))

l_perso.append(0)
l_drugs.append(0)

In [518]:
profil = data.iloc[:,0:6]
profil.set_index('ID',inplace = True)

perso = data.iloc[:,l_perso]
perso.set_index('ID',inplace = True)

drugs = data.iloc[:,l_drugs]
drugs.set_index('ID',inplace = True)

profil_jeunes = data_jeunes.iloc[:,0:6]
profil_jeunes.set_index('ID',inplace = True)

perso_jeunes = data_jeunes.iloc[:,l_perso]
perso_jeunes.set_index('ID',inplace = True)

drugs_jeunes = data_jeunes.iloc[:,l_drugs]
drugs_jeunes.set_index('ID',inplace = True)

In [519]:
drugs

,alcohol,amphet,amyl,benzos,caff,cannabis,choc,coke,crack,ecstasy,heroin,ketamine,legalhigh,LSD,meth,mushrooms,nicotine,semer,VSabuse
ID,,,,,,,,,,,,,,,,,,,
1,CL5,CL2,CL0,CL2,CL6,CL0,CL5,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
2,CL5,CL2,CL2,CL0,CL6,CL4,CL6,CL3,CL0,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
3,CL6,CL0,CL0,CL0,CL6,CL3,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
4,CL4,CL0,CL0,CL3,CL5,CL2,CL4,CL2,CL0,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
5,CL4,CL1,CL1,CL0,CL6,CL3,CL6,CL0,CL0,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,CL5,CL0,CL0,CL0,CL4,CL5,CL4,CL0,CL0,CL0,CL0,CL0,CL3,CL3,CL0,CL0,CL0,CL0,CL5
1885,CL5,CL0,CL0,CL0,CL5,CL3,CL4,CL0,CL0,CL2,CL0,CL0,CL3,CL5,CL4,CL4,CL5,CL0,CL0
1886,CL4,CL6,CL5,CL5,CL6,CL6,CL6,CL4,CL0,CL4,CL0,CL2,CL0,CL2,CL0,CL2,CL6,CL0,CL0


Décrivons en premier lieu les colonnes décrivant le profil de l'individu\
Les colonnes 'profil' :
- ID : Categorical variable
- Age : Interval variable 
- Genre : Categorical variable 
- Education : Ordinal Variable 
- Pays : Categorical Variable 
- Ethnicité : Categorical Variable 

Soit 6 colonnes.

In [520]:
for i in range(1,6) :
    print('La colonne \'',names_column[i],'\' a ',len(profil.iloc[:,i-1].unique()),' valeurs uniques ',data.iloc[:,i].unique(),'\n')
    


La colonne ' age ' a  6  valeurs uniques  [ 0.49788 -0.07854 -0.95197  2.59171  1.09449  1.82213] 

La colonne ' gender ' a  2  valeurs uniques  [ 0.48246 -0.48246] 

La colonne ' education ' a  9  valeurs uniques  [-0.05921  1.98437  1.16365 -1.22751 -1.7379   0.45468 -0.61113 -2.43591
 -1.43719] 

La colonne ' country ' a  7  valeurs uniques  [ 0.96082  0.24923 -0.57009 -0.28519 -0.09765  0.21128 -0.46841] 

La colonne ' ethnicity ' a  7  valeurs uniques  [ 0.126   -0.31685  0.1144  -0.22166 -0.50212 -1.10702  1.90725] 



Conclusion :
Nous pouvons observer que les colonnes 'profil' sont dans l'ensemble catégorique (même si à première vue, les valeurs étant numériques, nous aurions pu penser autrement) et donc que pour leur visualisation, des valeurs de type string seraient plus appropiées à priori.

---

Décrivons les colonnes décrivant la personnalité l'individu\
Les colonnes 'perso':

- les colonnes se basant sur le test NEO-FFI-R (neuroticism, extraversion, openness to experience, agreeableness, and conscientiousness),
- la colonne impulsivité selon le test BIS-11 (impulsivity), 
- la colonne de sensation selon le test ImpSS (sensation seeking)

Soit 7 colonnes.

In [521]:
for i in range(6,13) :
    print('La colonne \'',names_column[i],'\' a ',len(data.iloc[:,i].unique()),' valeurs uniques ',data.iloc[:,i].unique(),'\n')


La colonne ' nscore ' a  49  valeurs uniques  [ 0.31287 -0.67825 -0.46725 -0.14882  0.73545 -1.32828  0.62967 -0.24649
 -1.05308  2.28554 -0.79151 -0.92104 -2.05048 -1.55078  0.52135  1.37297
 -0.34799 -1.1943   0.41667  1.60383 -0.05188  0.04257  1.02119  0.82562
  0.13606  1.13281 -1.43907 -0.58016  0.22393 -2.21844  0.91093 -2.3436
  1.8399   1.49158  3.27393 -1.69163  1.98437  2.82196  1.23461 -1.86962
  2.127   -2.42317  2.46262  1.72012  2.61139 -2.75696 -2.52197 -3.15735
 -3.46436] 

La colonne ' escore ' a  42  valeurs uniques  [-0.57545  1.93886  0.80523 -0.80615 -1.6334  -0.30033 -1.09207  2.57309
  0.00332  0.16767  1.45421 -1.50796 -1.23177 -0.15487 -1.7625   0.47617
 -0.94779 -3.27393  0.63779 -0.43999  1.74091  0.32197  1.2861   0.96248
  1.11406 -0.69509  1.58487 -1.37639 -1.92173  3.00537  2.127    2.32338
 -2.21069 -2.32338 -2.11437 -2.72827 -2.5383  -2.44904 -2.03972  2.8595
  3.27393 -3.00537] 

La colonne ' oscore ' a  35  valeurs uniques  [-0.58331  1.43533 -0.8473

# <font color = 'red'>Arthur</font> -> description  des colonnes perso 

Conclusion : Nous pouvons observer que les colonnes 'perso' sont dans l'ensemble numériques (même si à première vue, les valeurs étant numériques, nous aurions pu penser autrement).

---

In [522]:
categorical_columns = (data.dtypes == 'object')
object_cols = list(categorical_columns[categorical_columns].index)

In [523]:
object_cols

['alcohol',
 'amphet',
 'amyl',
 'benzos',
 'caff',
 'cannabis',
 'choc',
 'coke',
 'crack',
 'ecstasy',
 'heroin',
 'ketamine',
 'legalhigh',
 'LSD',
 'meth',
 'mushrooms',
 'nicotine',
 'semer',
 'VSabuse']

La dangerosité de chaque drogue est classée selon ce site : 
https://www.frontiersin.org/files/Articles/592199/fpsyt-11-592199-HTML/image_m/fpsyt-11-592199-g001.jpg <br>
C'est un travail de recherche en amont mais le taux de dangerosité de chaque drogue sera prouvée plus tard lors de la phase de pondération, avec un autre jeu de données. Pour le moment, tout ce qui nous intéresse, c'est le fait qu'elle soit considérée comme possiblement dangereuse.
<br><br>
La liste des drogues considérées "dures" et "douces" ici :<br>
https://www.zamnesia.fr/blog-la-classification-des-drogues-dures-et-douces-n1425


Décrivons désormais les colonnes mettant en avant la fréquence de consommation de l'individu\
Les colonnes 'drugs' sont chacune caractéristiques d'une drogue:
- l'alcool : <font color = 'red'>3ème plus dangereuse</font> et <font color = 'red'>'dure'</font>
- l'amphétamine : <font color = 'red'>5ème plus dangereuse</font> et <font color = 'red'>'dure'</font>
- le nitrite d'amyl (utilisation des poppers) : <font color = 'yellow'>Pas considérée comme drogue dure</font> 
- le benzodiazepine : <font color = 'orange'>9ème plus dangereuse</font> et <font color = 'red'>'dure'</font>
- la caffeine : <font color ='green'>Pas considéré comme dangereuse</font>
- le cannabis : <font color = 'orange'>10ème plus dangereuse</font> mais <font color = 'green'>'douce'</font>
- le chocolat : <font color ='green'>Pas considéré comme dangereuse</font>
- la cocaine : <font color = 'red'>4ème plus dangereuse</font> et <font color = 'red'>'dure'</font>
- le crack : <font color = 'red'>1ère plus dangereuse</font> et <font color = 'red'>'dure'</font>
- l'ectasy : <font color = 'orange'>7ème plus dangereuse</font> et <font color = 'red'>'dure'</font>
- l'heroin : <font color = 'red'>2ème plus dangereuse</font> et <font color = 'red'>'dure'</font>
- la ketamine : <font color = 'orange'>8ème plus dangereuse</font> et <font color = 'red'>'dure'</font>
- les euphorisants légaux/Legal Highs (comme les cannabinoides synthétiques) : <font color = 'orange'>6ème plus dangereuse</font> mais <font color = 'green'>'douce'</font>
- le LSD : <font color = 'orange'>12ème plus dangereuse</font> et <font color = 'red'>'dure'</font>
- la methadone : <font color = 'orange'>14ème plus dangereuse</font> et <font color = 'red'>'dure'</font>
- les champignons magiques or magics Mushrooms : <font color = 'orange'>11ème plus dangereuse</font> et <font color = 'red'>'dure'</font>
- la nicotine : <font color = 'orange'>13ème plus dangereuse</font>
- la Semeron : <font color = 'yellow'>Pas considérée comme drogue dure et pas d'information sur sa dangerosité</font>
- Les substances volatiles (Abusement) : <font color = 'yellow'>Pas considérée comme drogue dure</font>

Soit 19 colonnes.

In [524]:
list(drugs.columns)

['alcohol',
 'amphet',
 'amyl',
 'benzos',
 'caff',
 'cannabis',
 'choc',
 'coke',
 'crack',
 'ecstasy',
 'heroin',
 'ketamine',
 'legalhigh',
 'LSD',
 'meth',
 'mushrooms',
 'nicotine',
 'semer',
 'VSabuse']

In [525]:
drugs.dtypes

alcohol      object
amphet       object
amyl         object
benzos       object
caff         object
cannabis     object
choc         object
coke         object
crack        object
ecstasy      object
heroin       object
ketamine     object
legalhigh    object
LSD          object
meth         object
mushrooms    object
nicotine     object
semer        object
VSabuse      object
dtype: object

In [526]:
drugs_drop_NoDangerous = ['amyl','caff','choc','semer','VSabuse'] # pas considérée dangereuse

drugs_danger = ['crack','heroin','alcohol','coke','amphet','legalhigh',
                'ecstasy','ketamine','benzos','cannabis','mushrooms',
                'LSD','nicotine','methadone']
drugs_danger_ranking = dict(zip(list(range(1,15)),drugs_danger))

drugs_douce = ['amyl','cannabis','legalhigh','semer','VSabuse','caff','choc']
drugs_dure = list(filter(lambda i:i not in drugs_douce,list(drugs.columns)))


In [527]:
drugs_dure

['alcohol',
 'amphet',
 'benzos',
 'coke',
 'crack',
 'ecstasy',
 'heroin',
 'ketamine',
 'LSD',
 'meth',
 'mushrooms',
 'nicotine']

In [528]:
drugs_danger_ranking

{1: 'crack',
 2: 'heroin',
 3: 'alcohol',
 4: 'coke',
 5: 'amphet',
 6: 'legalhigh',
 7: 'ecstasy',
 8: 'ketamine',
 9: 'benzos',
 10: 'cannabis',
 11: 'mushrooms',
 12: 'LSD',
 13: 'nicotine',
 14: 'methadone'}

Nous allons donner plutard une pondération  à chaque colonne selon sa dangerosité ! (Lors du modeling à l'aide du deuxième dataset)

Voici les valeurs unique issus de chaque colonne avec la fréquence de chaque valeur unique chez les jeunes :

In [529]:
drugs_frequency_jeunes_int = (round((drugs_jeunes.apply(pd.value_counts))*100/drugs_jeunes.shape[0],2))

In [530]:
drugs_frequency_all_int =  (round((drugs.apply(pd.value_counts))*100/drugs.shape[0],2))

In [531]:
drugs_frequency_jeunes = drugs_frequency_jeunes_int.copy().astype(str)

In [532]:
drugs_frequency_all = drugs_frequency_all_int.copy().astype(str)

In [533]:
def join(x) :
    return x + '%'


In [534]:
for el in list(drugs.columns) :
    drugs_frequency_jeunes[el] = drugs_frequency_jeunes[el].apply(join)
    drugs_frequency_all[el] = drugs_frequency_all[el].apply(join)


In [535]:
display(drugs_frequency_jeunes)
display(drugs_frequency_all)

# proportion des jeunes dans les fréquences

,alcohol,amphet,amyl,benzos,caff,cannabis,choc,coke,crack,ecstasy,heroin,ketamine,legalhigh,LSD,meth,mushrooms,nicotine,semer,VSabuse
CL0,2.18%,47.59%,75.58%,51.48%,0.31%,9.02%,1.09%,54.28%,88.34%,38.41%,84.6%,74.03%,29.39%,49.61%,66.1%,42.77%,16.64%,99.07%,72.01%
CL1,nan%,0.16%,0.31%,0.47%,0.16%,0.47%,0.16%,0.47%,0.16%,0.16%,nan%,0.31%,0.16%,0.47%,0.16%,0.16%,0.31%,0.31%,2.64%
CL2,2.02%,11.98%,12.75%,10.26%,1.56%,7.15%,0.31%,10.11%,4.98%,10.89%,4.51%,4.98%,14.31%,7.31%,5.91%,11.98%,7.47%,0.16%,14.0%
CL3,11.66%,19.91%,8.09%,16.95%,4.04%,12.44%,3.73%,21.0%,5.29%,26.75%,5.29%,12.44%,32.35%,23.33%,14.62%,28.15%,14.15%,0.31%,7.62%
CL4,18.82%,7.62%,2.33%,10.58%,8.55%,12.13%,21.31%,8.4%,0.62%,15.71%,2.64%,4.35%,11.51%,10.58%,4.67%,12.75%,10.58%,0.16%,1.56%
CL5,42.15%,5.91%,0.62%,4.51%,20.53%,17.73%,35.93%,4.04%,0.62%,5.6%,1.4%,3.58%,6.69%,7.31%,3.73%,4.2%,13.06%,nan%,1.4%
CL6,23.17%,6.84%,0.31%,5.75%,64.85%,41.06%,37.48%,1.71%,nan%,2.49%,1.56%,0.31%,5.6%,1.4%,4.82%,nan%,37.79%,nan%,0.78%


,alcohol,amphet,amyl,benzos,caff,cannabis,choc,coke,crack,ecstasy,heroin,ketamine,legalhigh,LSD,meth,mushrooms,nicotine,semer,VSabuse
CL0,1.8%,51.78%,69.23%,53.05%,1.43%,21.91%,1.7%,55.07%,86.31%,54.16%,85.15%,79.05%,58.04%,56.71%,75.81%,52.1%,22.71%,99.58%,77.19%
CL1,1.8%,12.2%,11.14%,6.15%,0.53%,10.98%,0.16%,8.49%,3.55%,5.99%,3.61%,2.39%,1.54%,13.74%,2.07%,11.09%,10.24%,0.11%,10.61%
CL2,3.61%,12.89%,12.57%,12.41%,1.27%,14.11%,0.53%,14.32%,5.94%,12.41%,4.99%,7.53%,10.5%,9.39%,5.15%,13.79%,10.82%,0.16%,7.16%
CL3,10.5%,10.5%,4.88%,12.52%,3.18%,11.19%,2.86%,13.69%,3.13%,14.69%,3.45%,6.84%,17.14%,11.35%,7.9%,14.59%,9.81%,0.11%,3.24%
CL4,15.23%,3.98%,1.27%,6.37%,5.62%,7.43%,15.7%,5.25%,0.48%,8.28%,1.27%,2.23%,5.84%,5.15%,2.65%,6.1%,5.73%,0.05%,0.69%
CL5,40.27%,3.24%,0.74%,4.46%,14.48%,9.81%,36.23%,2.18%,0.48%,3.34%,0.85%,1.75%,3.4%,2.97%,2.55%,2.12%,8.33%,nan%,0.74%
CL6,26.79%,5.41%,0.16%,5.04%,73.47%,24.56%,42.81%,1.01%,0.11%,1.11%,0.69%,0.21%,3.55%,0.69%,3.87%,0.21%,32.36%,nan%,0.37%


- Comme on pouvait l'imaginer, on peut observer que la plupart des drogues les plus "dures" sont les moins utilisées de par leur dangerosité. (<font color = 'green'>Précisions dans la Data Visualisation</font>)
- On peut également observer que CL1 (used over the last decade) n'est pas trop applicable dans le cas des jeunes et qu'on laissera ces cas isolés de côté. (<font color = 'green'>Précisions dans le Data Wrangling</font>)
- On peut remarquer aussi des pourcentages assez élévé en termes d'utilisation du cannabis chez les jeunes (<font color = 'green'>Précisions dans la Data Visualisation</font>)

In [536]:
for i in range(13,32) :
    print('La colonne \'',names_column[i],'\' a ',len(data.iloc[:,i].unique()),' valeurs uniques ',data.iloc[:,i].unique(),'\n')


La colonne ' alcohol ' a  7  valeurs uniques  ['CL5' 'CL6' 'CL4' 'CL2' 'CL1' 'CL0' 'CL3'] 

La colonne ' amphet ' a  7  valeurs uniques  ['CL2' 'CL0' 'CL1' 'CL3' 'CL5' 'CL4' 'CL6'] 

La colonne ' amyl ' a  7  valeurs uniques  ['CL0' 'CL2' 'CL1' 'CL3' 'CL5' 'CL4' 'CL6'] 

La colonne ' benzos ' a  7  valeurs uniques  ['CL2' 'CL0' 'CL3' 'CL1' 'CL4' 'CL5' 'CL6'] 

La colonne ' caff ' a  7  valeurs uniques  ['CL6' 'CL5' 'CL4' 'CL3' 'CL0' 'CL1' 'CL2'] 

La colonne ' cannabis ' a  7  valeurs uniques  ['CL0' 'CL4' 'CL3' 'CL2' 'CL1' 'CL6' 'CL5'] 

La colonne ' choc ' a  7  valeurs uniques  ['CL5' 'CL6' 'CL4' 'CL0' 'CL3' 'CL2' 'CL1'] 

La colonne ' coke ' a  7  valeurs uniques  ['CL0' 'CL3' 'CL2' 'CL1' 'CL6' 'CL5' 'CL4'] 

La colonne ' crack ' a  7  valeurs uniques  ['CL0' 'CL1' 'CL2' 'CL3' 'CL5' 'CL4' 'CL6'] 

La colonne ' ecstasy ' a  7  valeurs uniques  ['CL0' 'CL4' 'CL1' 'CL3' 'CL2' 'CL6' 'CL5'] 

La colonne ' heroin ' a  7  valeurs uniques  ['CL0' 'CL1' 'CL2' 'CL3' 'CL5' 'CL6' 'CL4'] 

La c

- CL0 : Never Used  (0)
- CL1 : Used over a decade (1)
- CL2 : Used in Last Decade (2)
- CL3 : Used in Last Year (3)
- CL4 : Used in Last month (4)
- CL5 : Used Last week (5)
- CL6 : Used in Last day (6)

---

## <p style = 'text-align: center;'>Data Cleaning and Wrangling</p> 

---

Cette partie sera dédiée au nettoyage (renommer les colonnes,reset des index, drop des colonnes, ) et à la mise en forme de la donnée pour que cette dernière soit prête à l'exploitation

In [537]:
data.head()

,ID,age,gender,education,country,ethnicity,nscore,escore,oscore,ascore,...,ecstasy,heroin,ketamine,legalhigh,LSD,meth,mushrooms,nicotine,semer,VSabuse
0,1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
1,2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,...,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
2,3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,...,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
3,4,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,...,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
4,5,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,...,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0


#### Nous allons commencer par drop les colonnes qui ne nous intéressent pas.
Il y a deux types de colonnes que nous allons drop :
- celles qui n'ont pas d'intérêt dans le sujet (Par exemple, comme nous travaillons avec les drogues dures, le chocolat n'a rien à faire ici)
- celles qui n'ont pas d'intérêt dans le dataset (Par exemple, si la fréquence de C0 est très grande pour les drogues, ce qui veut dire que ce n'est pas des drogues utilisées ou les Outliers)

<font color = 'red'>ARTHUR -> colonnes personnalité to drop</font>

In [538]:
# initialisons la liste qui contiendra les noms de colonnes à drop
to_drop = []

In [539]:
# fréquence de CO trop grande 
drugs_frequency_used = drugs_frequency_jeunes_int.reset_index(drop = True).iloc[0]

In [540]:
drugs_frequency_used.info()

<class 'pandas.core.series.Series'>
Index: 19 entries, alcohol to VSabuse
Series name: 0
Non-Null Count  Dtype  
--------------  -----  
19 non-null     float64
dtypes: float64(1)
memory usage: 860.0+ bytes


In [541]:
drugs_frequency_used[drugs_frequency_used >= 90] # 90

semer    99.07
Name: 0, dtype: float64

La seule drogue qui n'est quasi pas utilisée par les jeunes est le semeron et elle fait déjà partie des drogues considérées comme 'non dure'.

In [542]:
# les colonnes 'profil' sont toutes plus ou moins utiles à part 'ID'
# qui nous sert plus à identifier chaque ligne quelque soit le changement fait sur les index

# les colonnes 'perso

# les colonnes 'drogues' ne sont pas toutes utiles

#
to_drop.extend(drugs_drop_NoDangerous) # pas d'intérêt dans la problématique

In [543]:
data.drop(to_drop,inplace = True, axis = 1)

In [544]:
to_drop.append('age')

In [545]:
data_jeunes.drop(to_drop,inplace = True, axis = 1) 
# on drop age dans ce jeu de données mais pas dans le 1er comme la colonne n'a qu'une seule valeur unique


C:\Users\rayan\AppData\Local\Temp\ipykernel_5996\2499484773.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_jeunes.drop(to_drop,inplace = True, axis = 1)


In [546]:
to_drop

['amyl', 'caff', 'choc', 'semer', 'VSabuse', 'age']

In [547]:
display(data.head(3))
display(data_jeunes.head(3))

,ID,age,gender,education,country,ethnicity,nscore,escore,oscore,ascore,...,coke,crack,ecstasy,heroin,ketamine,legalhigh,LSD,meth,mushrooms,nicotine
0,1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2
1,2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,...,CL3,CL0,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4
2,3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0


,ID,gender,education,country,ethnicity,nscore,escore,oscore,ascore,csore,...,coke,crack,ecstasy,heroin,ketamine,legalhigh,LSD,meth,mushrooms,nicotine
3,4,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,...,CL2,CL0,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2
33,34,-0.48246,0.45468,0.96082,-0.31685,0.41667,-0.30033,0.29338,1.28610,-0.14277,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0
154,155,0.48246,-0.61113,0.96082,-0.31685,2.82196,-1.37639,0.72330,-0.45321,0.58489,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0


#### Après avoir étudié les colonnes, nous passons aux index :

In [548]:
first_row = data.iloc[0]
last_row = data.iloc[-1]

In [549]:
pd.DataFrame(first_row).T

,ID,age,gender,education,country,ethnicity,nscore,escore,oscore,ascore,...,coke,crack,ecstasy,heroin,ketamine,legalhigh,LSD,meth,mushrooms,nicotine
0,1,0.49788,0.48246,-0.05921,0.96082,0.126,0.31287,-0.57545,-0.58331,-0.91699,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2


In [550]:
pd.DataFrame(last_row).T

,ID,age,gender,education,country,ethnicity,nscore,escore,oscore,ascore,...,coke,crack,ecstasy,heroin,ketamine,legalhigh,LSD,meth,mushrooms,nicotine
1884,1888,-0.95197,-0.48246,-0.61113,0.21128,-0.31685,-0.46725,2.127,1.65653,1.11406,...,CL3,CL0,CL3,CL0,CL0,CL3,CL3,CL0,CL3,CL6


In [551]:
display(data.head(2))
display(data.tail(2))

,ID,age,gender,education,country,ethnicity,nscore,escore,oscore,ascore,...,coke,crack,ecstasy,heroin,ketamine,legalhigh,LSD,meth,mushrooms,nicotine
0,1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2
1,2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,...,CL3,CL0,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4


,ID,age,gender,education,country,ethnicity,nscore,escore,oscore,ascore,...,coke,crack,ecstasy,heroin,ketamine,legalhigh,LSD,meth,mushrooms,nicotine
1883,1887,-0.95197,0.48246,-0.61113,-0.57009,-0.31685,0.91093,-1.92173,0.29338,-1.62090,...,CL0,CL0,CL3,CL0,CL0,CL3,CL3,CL0,CL3,CL4
1884,1888,-0.95197,-0.48246,-0.61113,0.21128,-0.31685,-0.46725,2.12700,1.65653,1.11406,...,CL3,CL0,CL3,CL0,CL0,CL3,CL3,CL0,CL3,CL6


On peut voir qu'il ya un décalage étrange entre l'écart initial de l'ID et de l'index aux premières lignes et l'écart qu'on voit aux dernières lignes.\
Nous allons donc voir à quel niveau l'aggrandissement de l'écart occurre en comparant les valeurs index et ID

In [552]:
# On commence par décaler les valeurs de la colonne ID 
# afin qu'elle puisse matcher avec les valeurs des index

data['ID'].index = data['ID'].index + 1 
Data_ID = pd.concat([pd.Series([0]),data['ID'][:-1]])
Data_ID

0          0
1          1
2          2
3          3
4          4
        ... 
1880    1883
1881    1884
1882    1885
1883    1886
1884    1887
Length: 1885, dtype: int64

In [553]:
# et on compare
pd.Series(data.index).compare(Data_ID)

,self,other
294,294.0,295.0
295,295.0,296.0
296,296.0,297.0
297,297.0,298.0
298,298.0,300.0
...,...,...
1880,1880.0,1883.0
1881,1881.0,1884.0
1882,1882.0,1885.0
1883,1883.0,1886.0


Avec cette comparaison nous pouvons affirmer que le décalage a commencé avec la ligne 294.

Vérifions qu'il n y ait pas d'anomalie aux alentours de cette ligne.

In [554]:
data.iloc[290:296]

,ID,age,gender,education,country,ethnicity,nscore,escore,oscore,ascore,...,coke,crack,ecstasy,heroin,ketamine,legalhigh,LSD,meth,mushrooms,nicotine
290,291,0.49788,0.48246,-0.05921,0.96082,-1.10702,-0.92104,0.96248,-0.84732,1.45039,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0
291,292,0.49788,0.48246,-0.61113,0.96082,-0.31685,0.62967,-1.09207,0.44585,-0.76096,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL4
292,293,1.09449,0.48246,-0.05921,0.96082,-0.31685,-0.05188,-0.15487,-0.58331,-0.60633,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0
293,295,-0.07854,0.48246,-0.05921,0.96082,-0.31685,1.13281,0.00332,-0.17779,-0.76096,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6
294,296,-0.95197,0.48246,-1.43719,0.96082,-0.31685,-0.34799,0.47617,-0.17779,-0.60633,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0
295,297,0.49788,-0.48246,-0.05921,0.96082,-0.31685,1.37297,-0.43999,0.14143,-0.01729,...,CL0,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL0,CL2


Aucune anomalie à priori outre le fait qu"il manque l'individu dont l'ID est 294 et pas seulement vu l'écart qu'on a observé un peu plus haut. ( 3 ID manquant dont le premier est 294)


Cependant peut être qu'une anomalie n'est pas directement visible mais afin de donner une <font color = 'cyan'>unicité à chaque ligne</font> et <font color =cyan>ne pas avoir deux valeurs différentes pour la caractériser</font>, nous allons utiliser les valeurs de la colonne ID comme index !

Vérifions que chaque valeur de la colonne est unique 

In [555]:
data['ID'].is_unique

True

In [556]:
data.set_index('ID',inplace = True)

In [557]:
data_jeunes.set_index('ID',inplace = True)

In [558]:
display(data.head(5))
display(data_jeunes.head(5))

,age,gender,education,country,ethnicity,nscore,escore,oscore,ascore,csore,...,coke,crack,ecstasy,heroin,ketamine,legalhigh,LSD,meth,mushrooms,nicotine
ID,,,,,,,,,,,,,,,,,,,,,
1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2
2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,-0.14277,...,CL3,CL0,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4
3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0
4,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,...,CL2,CL0,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2
5,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,1.30612,...,CL0,CL0,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2


,gender,education,country,ethnicity,nscore,escore,oscore,ascore,csore,impulsive_bis11,...,coke,crack,ecstasy,heroin,ketamine,legalhigh,LSD,meth,mushrooms,nicotine
ID,,,,,,,,,,,,,,,,,,,,,
4,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,...,CL2,CL0,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2
34,-0.48246,0.45468,0.96082,-0.31685,0.41667,-0.30033,0.29338,1.28610,-0.14277,-0.71126,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0
155,0.48246,-0.61113,0.96082,-0.31685,2.82196,-1.37639,0.72330,-0.45321,0.58489,-0.21712,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0
167,-0.48246,-0.61113,0.96082,-0.31685,1.49158,-1.23177,0.72330,-2.35413,-1.92173,0.88113,...,CL3,CL3,CL0,CL0,CL0,CL2,CL0,CL0,CL3,CL6
170,0.48246,-1.22751,0.96082,-0.31685,-0.34799,0.00332,-1.42424,0.59042,-0.14277,-0.71126,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL4


In [559]:
del first_row['ID']

In [560]:
# on vérifie
first_row.equals(data.loc[1])
# Si c'est True, c'est que le set_index() s'est bien déroulée

True

In [561]:
display(pd.DataFrame(data.loc[1]).T)
display(pd.DataFrame(first_row).T)

,age,gender,education,country,ethnicity,nscore,escore,oscore,ascore,csore,...,coke,crack,ecstasy,heroin,ketamine,legalhigh,LSD,meth,mushrooms,nicotine
1,0.49788,0.48246,-0.05921,0.96082,0.126,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2


,age,gender,education,country,ethnicity,nscore,escore,oscore,ascore,csore,...,coke,crack,ecstasy,heroin,ketamine,legalhigh,LSD,meth,mushrooms,nicotine
0,0.49788,0.48246,-0.05921,0.96082,0.126,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2


#### Nous allons nettoyer désormais les colonnes :
- Vérifier qu'il n y a pas plusieurs types de valeurs par colonne 
- Mapping

In [562]:
data.dtypes

age                float64
gender             float64
education          float64
country            float64
ethnicity          float64
nscore             float64
escore             float64
oscore             float64
ascore             float64
csore              float64
impulsive_bis11    float64
sensation_ImpSS    float64
alcohol             object
amphet              object
benzos              object
cannabis            object
coke                object
crack               object
ecstasy             object
heroin              object
ketamine            object
legalhigh           object
LSD                 object
meth                object
mushrooms           object
nicotine            object
dtype: object

Après inspection :
-  on peut voir que les colonnes 'drugs' sont de type Object. Ce dtype est capable d'englober tout type de data à la fois(int + float + str etc ). On doit donc vérifier leur composition.
- Des variables ayant très peu de valeurs uniques (les variables 'profils') sont des float64 alors que ce type est indicateur de variables numériques
- Les variales de personnalité ont des types qui à priori leur correspondent, à voir.


Commençons par inspecter les variables de 'drugs'

In [563]:
# comme des colonnes ont été dropd ans cette section, nous allons mettre à jour la dataframe 'drugs'ArithmeticError
drugs = pd.merge(data,drugs,on = 'alcohol', how = 'right')

In [564]:
drugs.dtypes

age                float64
gender             float64
education          float64
country            float64
ethnicity          float64
nscore             float64
escore             float64
oscore             float64
ascore             float64
csore              float64
impulsive_bis11    float64
sensation_ImpSS    float64
alcohol             object
amphet_x            object
benzos_x            object
cannabis_x          object
coke_x              object
crack_x             object
ecstasy_x           object
heroin_x            object
ketamine_x          object
legalhigh_x         object
LSD_x               object
meth_x              object
mushrooms_x         object
nicotine_x          object
amphet_y            object
amyl                object
benzos_y            object
caff                object
cannabis_y          object
choc                object
coke_y              object
crack_y             object
ecstasy_y           object
heroin_y            object
ketamine_y          object
l

#### Data Wrangling :

- S'aider de l'exploration en première partie et l'approfondir
- S'occuper des N/A s'il y en a
- Trouver les Outliers
- Remettre en forme la data (mapping)
- Filtrer la data davantage 

N/A ? 

In [570]:
data.isna().sum()

age                0
gender             0
education          0
country            0
ethnicity          0
nscore             0
escore             0
oscore             0
ascore             0
csore              0
impulsive_bis11    0
sensation_ImpSS    0
alcohol            0
amphet             0
benzos             0
cannabis           0
coke               0
crack              0
ecstasy            0
heroin             0
ketamine           0
legalhigh          0
LSD                0
meth               0
mushrooms          0
nicotine           0
dtype: int64

- Bon point : Nous n'avons aucune valeur nulle dans le tableau

Mapping

In [588]:
data["age"].map(map_dict_age)
data["gender"].map(map_dict_gender)
data["education"].map(map_dict_education)
data["country"].map(map_dict_country)
data["ethnicity"].map(map_dict_ethnicity)

data.iloc[:,12:].apply(map(map_dict_drugs))

AttributeError: 'DataFrame' object has no attribute 'map'

In [593]:
data.iloc[:,12:].applymap(map_dict_drugs.get)

,alcohol,amphet,benzos,cannabis,coke,crack,ecstasy,heroin,ketamine,legalhigh,LSD,meth,mushrooms,nicotine
ID,,,,,,,,,,,,,,
1,5,2,2,0,0,0,0,0,0,0,0,0,0,2
2,5,2,0,4,3,0,4,0,2,0,2,3,0,4
3,6,0,0,3,0,0,0,0,0,0,0,0,1,0
4,4,0,3,2,2,0,0,0,2,0,0,0,0,2
5,4,1,0,3,0,0,1,0,0,1,0,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1884,5,0,0,5,0,0,0,0,0,3,3,0,0,0
1885,5,0,0,3,0,0,2,0,0,3,5,4,4,5
1886,4,6,5,6,4,0,4,0,2,0,2,0,2,6


Comme nous avons pu le remarquer dans l'exploration de données, certains individus de 18 à 24 ans ont des valeurs CL1 dans leur fréquence de consommation de drogues.<br><br>
Cependant CL1 équivaut à <==> la drogue a été consommée il y a plus d'une décennie.<br><br>
En prenant en compte cette information (une consommation de drogue avant 13 ans n'est pas de l'ordre de notre analyse) et la fréquence dérisoire d'individus jeunes possédant une CL1 nous poussent donc à remplacer ces valeurs par CLO pour l'ensemble des jeunes.

In [566]:
(drugs_frequency_jeunes_int.loc['CL1']).mean()
# la fréquence moyenne des valeurs CL1 est de 0.41% pour les jeunes 
# cette possibilité est représentative est donc isolée

0.4141176470588235

In [ ]:
data = data.where(data[data['age'] == '])

In [567]:
# on clean aussi les dataframes utilisés en raccourci

data_jeunes = data_jeunes.where(data_jeunes != 'CL1',other = 'CL0')
drugs_jeunes = drugs_jeunes.where(drugs_jeunes != 'CL1',other = 'CL0')

In [568]:
round((drugs_jeunes.apply(pd.value_counts))*100/drugs_jeunes.shape[0],2)
# on peut voir que CL1  a disparu

,alcohol,amphet,amyl,benzos,caff,cannabis,choc,coke,crack,ecstasy,heroin,ketamine,legalhigh,LSD,meth,mushrooms,nicotine,semer,VSabuse
CL0,2.18,47.74,75.89,51.94,0.47,9.49,1.24,54.74,88.49,38.57,84.60,74.34,29.55,50.08,66.25,42.92,16.95,99.38,74.65
CL2,2.02,11.98,12.75,10.26,1.56,7.15,0.31,10.11,4.98,10.89,4.51,4.98,14.31,7.31,5.91,11.98,7.47,0.16,14.00
CL3,11.66,19.91,8.09,16.95,4.04,12.44,3.73,21.00,5.29,26.75,5.29,12.44,32.35,23.33,14.62,28.15,14.15,0.31,7.62
CL4,18.82,7.62,2.33,10.58,8.55,12.13,21.31,8.40,0.62,15.71,2.64,4.35,11.51,10.58,4.67,12.75,10.58,0.16,1.56
CL5,42.15,5.91,0.62,4.51,20.53,17.73,35.93,4.04,0.62,5.60,1.40,3.58,6.69,7.31,3.73,4.20,13.06,NaN,1.40
CL6,23.17,6.84,0.31,5.75,64.85,41.06,37.48,1.71,NaN,2.49,1.56,0.31,5.60,1.40,4.82,NaN,37.79,NaN,0.78


---

## <p style = 'text-align: center;'>Visualisation</p> 

---

Legitimité de notre DataSet ? 
- Proportion des jeunes dans notre échantillon 

In [569]:
testtest = data.groupby(by = "age").count().reset_index()
testtest
plt.bar(testtest["age"], testtest["ID"])

KeyError: 'ID'

Health : Polyconsommation VS Consommation (quelle pondération ? )
